# AUV 3D

In [1]:
import sys
sys.path.append("../src/dynamics")
from pontryagin import system
from sympy import init_printing
from sympy import *
init_printing()

In [2]:
# state
x, y, z, vx, vy, vz, qr, qx, qy, qz = symbols('x y z v_x v_y v_z q_r q_x q_y q_z', real=True)
p                      = Matrix([x, y, z])
v                      = Matrix([vx, vy, vz])
q                      = Matrix([qr, qx, qy, qz])
s                      = Matrix([p, v, q])
s.T

[x  y  z  vₓ  v_y  v_z  qᵣ  qₓ  q_y  q_z]

In [3]:
# controls: throttle, reaction wheels
ut = symbols('u', real=True, nonnegative=True)
ux, uy = symbols("u_x u_y", real=True)
u = Matrix([ut, ux, uy])
u.T

[u  uₓ  u_y]

In [4]:
# physical parametres
mass, density, CD, planaform, thrust, rotation = symbols("m rho cd A T omega", real=True, positive=True)

# fluid velocity
vfx, vfy, vfz = symbols("v_xf v_yf v_zf", real=True)
vf = Matrix([vfx, vfy, vfz])

# optimisation parametres
alpha = symbols("alpha", real=True, nonnegative=True)

In [5]:
# free stream velocity
vinf = v - vf

# free stream velocity magnitude
vinfmag = sqrt(sum([var**2 for var in vinf]))

# fluid drag
Fd = - Rational(1,2)*density*vinf*vinfmag*CD*planaform

# thrust direction
ix = 2*(qx*qz - qy*qr)
iy = 2*(qy*qz - qx*qr)
iz = 1 - 2*(qx**2 + qy**2)
i  = Matrix([ix, iy, iz])

# thrust
Ft = thrust*ut*i

In [6]:
# translation
dp = v
dv = (Fd + Ft)/mass

# rotation
dq = rotation*Rational(1,2) * Matrix([
    [0, 0, -uy, ux],
    [0, 0, ux, uy],
    [uy, -ux, 0, 0],
    [-ux, -uy, 0, 0]
]) * q


# state transition
ds = Matrix([dp, dv, dq])
ds

⎡                                                 vₓ                          
⎢                                                                             
⎢                                                v_y                          
⎢                                                                             
⎢                                                v_z                          
⎢                                                                             
⎢                        ______________________________________________       
⎢                       ╱            2               2               2        
⎢  A⋅cd⋅ρ⋅(vₓ - v_xf)⋅╲╱  (vₓ - v_xf)  + (v_y - v_yf)  + (v_z - v_zf)         
⎢- ──────────────────────────────────────────────────────────────────── + T⋅u⋅
⎢                                   2                                         
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                 m 

In [7]:
# Lagrangian
L = (1-alpha)*(ut**2 + ux**2 + uy**2) + alpha*(ut + ux**2 + uy**2)
L

  ⎛      2      2⎞            ⎛ 2     2      2⎞
α⋅⎝u + uₓ  + u_y ⎠ + (-α + 1)⋅⎝u  + uₓ  + u_y ⎠

In [8]:
# initialise system and make KKT equations
sys = system(s, u, ds, L, inequality=None)
sys.KKT()
sys.KKTeqs

⎡                                                                         ⎛   
⎢T⋅λᵥ ₓ⋅(-2⋅qᵣ⋅q_y + 2⋅qₓ⋅q_z)   T⋅λ_v_y⋅(-2⋅qᵣ⋅qₓ + 2⋅q_y⋅q_z)   T⋅λ_v_z⋅⎝- 2
⎢───────────────────────────── + ────────────────────────────── + ────────────
⎣              m                               m                              

   2        2    ⎞                                                            
⋅qₓ  - 2⋅q_y  + 1⎠                              λ_q_r⋅ω⋅q_z   λ_q_x⋅ω⋅q_y   λ_
────────────────── + α + 2⋅u⋅(-α + 1), 2⋅α⋅uₓ + ─────────── + ─────────── - ──
  m                                                  2             2          

                                                                              
q_y⋅ω⋅qₓ   λ_q_z⋅ω⋅qᵣ                            λ_q_r⋅ω⋅q_y   λ_q_x⋅ω⋅q_z   λ
──────── - ────────── + 2⋅uₓ⋅(-α + 1), 2⋅α⋅u_y - ─────────── + ─────────── + ─
  2            2                                      2             2         

                                       ⎤
_q_y⋅ω⋅q

In [9]:
# look at solutions
sol = solve(sys.KKTeqs, sys.KKTvars, dict=True)

In [10]:
# assign the optimal controls
sys.uo = Matrix([sol[0][var] for var in sys.u])
sys.uo

⎡                                                                             
⎢-T⋅λᵥ ₓ⋅qᵣ⋅q_y + T⋅λᵥ ₓ⋅qₓ⋅q_z - T⋅λ_v_y⋅qᵣ⋅qₓ + T⋅λ_v_y⋅q_y⋅q_z - T⋅λ_v_z⋅qₓ
⎢                                                                             
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                  m⋅(α - 1)                  
⎢                                                                             
⎢                              ω⋅(-λ_q_r⋅q_z - λ_q_x⋅q_y + λ_q_y⋅qₓ + λ_q_z⋅qᵣ
⎢                              ───────────────────────────────────────────────
⎢                                                     4                       
⎢                                                                             
⎢                               ω⋅(λ_q_r⋅q_y - λ_q_x⋅q_z - λ_q_y⋅qᵣ + λ_q_z⋅qₓ
⎢                               ──────────────────────────────────────────────
⎣                                                   

In [11]:
sys.codegen("../src/dynamics/auv3d")